# Single AI crypto concept

In [ ]:
from xdata_config import *

## 1 - Imports and fonctions

In [ ]:
from functions_module import *

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.constraints import MaxNorm
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense 
from keras.models import load_model

## Special list if needed

In [ ]:
Binance_USDT_HALAL.index("ROSE/USDT")

In [ ]:
from datetime import date
TICKERS = "../Binance-Fast-Trade-Bot/volatile_volume_" + str(date.today()) + ".txt"
VOLATILE_COINS=[line.strip() for line in open(TICKERS)]
PAIR_WITH="BUSD"
VOLATILE_USDT_PAIRS=[coin+"/USDT" for coin in VOLATILE_COINS]
VOLATILE_BUSD_PAIRS=[coin+"/BUSD" for coin in VOLATILE_COINS]
VOLATILE_BUSD_PAIRS


In [ ]:
coins_to_download=''
for coin in VOLATILE_COINS:
    coins_to_download=coins_to_download+" "+coin
os.system(f"node database/ddargs.js {coins_to_download} {PAIR_WITH}")#node database/ddargs.js ORN BUSD

In [ ]:
pair_list = VOLATILE_BUSD_PAIRS
#tf = '1m'
oldest_pair = "BTC/USDT"
if oldest_pair not in pair_list: pair_list.append(oldest_pair)
df_list1m = {}
df_list1d = {}
df_list1h = {}
df_list5m = {}
df_list15m = {}


for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '1m', path="./database/")
    df_list1m[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '1d', path="./database/")
    df_list1d[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '1h', path="./database/")
    df_list1h[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '5m', path="./database/")
    df_list5m[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(
        ccxt.binance(), pair, '15m', path="./database/")
    df_list15m[pair] = df.loc[:]
del(df)
df_list = df_list1m
prerr("Data load 100% use df_list1d[\"BTC/USDT\"] for exemple to access")



## Checking the import

In [ ]:
# chking import
MetaData

In [ ]:
if BUY_MODE=="BUY_ONLY":
    buy_function=buy_up_only
elif BUY_MODE=="BUY_UP":
    buy_function=buy_up
elif  BUY_MODE=="BUY_DIP":
    buy_function=buy_min_up
elif  BUY_MODE=="AFTER_DEPTH":
    buy_function=buy_after_depth
elif  BUY_MODE=="BUY_UP_CLOSE":
    buy_function=buy_up_close

    

try:
    os.mkdir(DATA_DIR, mode = 0o777)
except Exception as e:
    print(e)
print(f"Results dir: {DATA_DIR}")



In [ ]:
xdf=pd.DataFrame()
count=0
row_numbers=50000
for pair in pair_list:
    if pair != "BTC/USDT":
        print("working on: "+pair ,end=" -->")
        try:
            df=mini_expand4(pair=pair,i=0,j=len(df_list1m[pair]),window=WINDOW_SIZE,metadata=MetaData,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,buy_function=buy_function)
            print("df original shape "+str(df.shape))
            print(f"df original shape buy mean : {df.buy.mean()*100}")
            df=df.reset_index()
            try:df.pop("num_index")
            except: pass
            try:df.pop("index")
            except: pass
            try:df.pop("date")
            except: pass
            df=data_shufler(df)            
            #df=data_chooser(df,weight=50,row_numbers=df.buy.sum()*2)
            df=data_chooser50(df,row_numbers=row_numbers)
            gc.collect()
            df=data_cleanup(df)
            df=df.dropna()
            print(pair+f" is processed -- {count}/{len(pair_list)}")
        except Exception as e:
            print(f"error while processing {pair} {count}/{len(pair_list)}")
            print(e)
        xdf=pd.concat([xdf,df],axis=0)
        count+=1
        del(df)
        gc.collect()
df=xdf
del xdf
gc.collect()


In [ ]:
df=df.reset_index().drop(columns="num_index")
gc.collect()
for i in range(1):
    df = df.reindex(np.random.permutation(df.index)).reset_index().drop(columns="index")
    gc.collect()
df

In [ ]:
df.to_feather(f"../Data/fea/w{WINDOW_SIZE}_buy{BUY_PERCENT}.fea")

In [ ]:
gc.collect()
print("df choosen data shape"+str(df.shape))
print(f"pair: {(df.shape[0]/2)==df.buy.sum()}")
dt=df.to_numpy(dtype=np.float32)
#dt=df.to_numpy()
dt=np.nan_to_num(dt,nan=0)
#dt=dt.astype(np.float32)
dt=np.nan_to_num(dt, neginf=0) 
dt=np.nan_to_num(dt, posinf=0) 

## normalisation
index_20percent= int(0.2*len(dt[:,0]))
print(index_20percent)
if True:
    if True:
    #if True:
        print("normalizing ...")
        mean = dt[index_20percent:, 0:-1].mean(axis=0)
        std = dt[index_20percent:, 0:-1].std(axis=0)



        dt[index_20percent:, 0:-1] -= mean 
        dt[index_20percent:, 0:-1] /= std

        dt[:index_20percent, :-1] -=mean
        dt[:index_20percent, :-1] /= std
        FIRST_NORM_FLAG=False
        ######################### SAVIN NORM ################
        try:
            Normalization={"mean":mean.tolist(),"std":std.tolist()}
            with open(Normalization_File, 'w+') as fp:
                        json.dump(Normalization, fp,  indent=4)
                        print(fp.name)
        except Exception as e:
            print("error Normalization in juppiter")
            print(e)
    else:print("already normalized")
    
    


In [ ]:
dt=np.nan_to_num(dt,nan=0)
dt=np.nan_to_num(dt, neginf=0) 
dt=np.nan_to_num(dt, posinf=0) 
dt=dt.astype(np.float32)
gc.collect()


In [ ]:

## Model
IN_DIM=dt.shape[1]-1
model = Sequential()
model.add(Dense(int(250),input_dim=IN_DIM,activation='relu')) 
model.add(Dropout(0.2))
model.add(Dense(int(20),activation='relu')) 
model.add(Dense(int(50),activation='relu')) 
model.add(Dense(1,activation='sigmoid'))
print(model.summary())
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
callbacks_a = ModelCheckpoint(filepath =Model_FileName,monitor ='val_accuracy',save_best_only = True, save_weights = True)
callbacks_b = EarlyStopping(monitor ='val_accuracy',mode='auto',patience=15,verbose=1)
print("saving file in: "+Model_FileName)
history = model.fit(dt[index_20percent:, 0:-1],
                dt[index_20percent:,-1],
                validation_data=(dt[:index_20percent, :-1],dt[:index_20percent,-1]),
                epochs=6000,
                batch_size=256*10,
                callbacks=[callbacks_a,callbacks_b])

print('##########################################################################')
print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")
print(Normalization_File)
print(Model_FileName)


In [ ]:
model1=model
model1.compile()
#model1.save(Model_FileName+"_v1.h5")
print(Normalization_File)
print(Model_FileName+"_v1.h5")



## 2- Model Plus

In [ ]:
#model1=model
## Model 2

IN_DIM=dt.shape[1]-1
model = Sequential()
model.add(Dense(int(2500),input_dim=IN_DIM,activation='relu')) 
model.add(Dropout(0.3))
model.add(Dense(int(600),activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(int(200),activation='relu')) 
model.add(Dropout(0.2))
model.add(Dense(int(100),activation='sigmoid')) 
model.add(Dropout(0.3))
model.add(Dense(int(600),activation='relu')) 
model.add(Dropout(0.2))
model.add(Dense(int(100),activation='sigmoid')) 

model.add(Dense(int(50),activation='relu'))
model.add(Dense(1,activation='sigmoid'))
print(model.summary())
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
callbacks_a = ModelCheckpoint(filepath =Model_FileName,monitor ='val_accuracy',save_best_only = True, save_weights = True)
callbacks_b = EarlyStopping(monitor ='val_accuracy',mode='auto',patience=8,verbose=1)
print("saving file in: "+Model_FileName)
history = model.fit(dt[index_20percent:, 0:-1],
                dt[index_20percent:,-1],
                validation_data=(dt[:index_20percent, :-1],dt[:index_20percent,-1]),
                epochs=500,
                batch_size=256*10,
                callbacks=[callbacks_a,callbacks_b])

print('##########################################################################')
print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")
print(Normalization_File)
print(Model_FileName)
#1735/1735 [==============================] - 1829s 1s/step - loss: 0.6198 - accuracy: 0.6550 - val_loss: 0.6129 - val_accuracy: 0.6633


In [ ]:
#model1=model
## Model 2

IN_DIM=dt.shape[1]-1
model = Sequential()
model.add(Dense(int(250),input_dim=IN_DIM,activation='relu')) 
model.add(Dropout(0.3))
model.add(Dense(int(60),activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(int(20),activation='relu')) 
model.add(Dropout(0.2))
model.add(Dense(int(10),activation='sigmoid')) 
model.add(Dropout(0.3))
model.add(Dense(int(60),activation='relu')) 
model.add(Dropout(0.2))
model.add(Dense(int(60),activation='sigmoid')) 

model.add(Dense(int(5),activation='relu'))
model.add(Dense(1,activation='sigmoid'))
print(model.summary())
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
callbacks_a = ModelCheckpoint(filepath =Model_FileName,monitor ='val_accuracy',save_best_only = True, save_weights = True)
callbacks_b = EarlyStopping(monitor ='val_accuracy',mode='auto',patience=8,verbose=1)
print("saving file in: "+Model_FileName)
history = model.fit(dt[index_20percent:, 0:-1],
                dt[index_20percent:,-1],
                validation_data=(dt[:index_20percent, :-1],dt[:index_20percent,-1]),
                epochs=500,
                batch_size=256*50,
                callbacks=[callbacks_a,callbacks_b])

print('##########################################################################')
print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")
print(Normalization_File)
print(Model_FileName)
#1735/1735 [==============================] - 95s 54ms/step - loss: 0.6314 - accuracy: 0.6439 - val_loss: 0.6263 - val_accuracy: 0.6563 batch_size=256*10
#8672/8672 [==============================] - 121s 14ms/step - loss: 0.6279 - accuracy: 0.6485 - val_loss: 0.6257 - val_accuracy: 0.6489  batch_size=256*2
#347/347 [==============================] - 77s 219ms/step - loss: 0.6540 - accuracy: 0.6301 - val_loss: 0.6353 - val_accuracy: 0.6522 batch_size=256*50


In [133]:
# CHATGPT Optimization
class_weights = {0: 10., 1: 1.}

IN_DIM = dt.shape[1] - 1
model = Sequential()
model.add(Dense(int(2500), input_dim=IN_DIM, activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(int(600), activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(int(200), activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(int(100), activation='sigmoid'))
model.add(Dense(int(600), activation='elu'))
model.add(BatchNormalization())
model.add(Dense(int(50), activation='elu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
callbacks_a = ModelCheckpoint(filepath=Model_FileName, monitor='val_accuracy', save_best_only=True, save_weights=True)
callbacks_b = EarlyStopping(monitor='val_accuracy', mode='auto', patience=8, verbose=1)
print("saving file in: " + Model_FileName)
history = model.fit(dt[index_20percent:, :-1],
                    dt[index_20percent:, -1],
                    validation_data=(dt[:index_20percent, :-1], dt[:index_20percent, -1]),
                    epochs=500,
                    batch_size=256 * 20,
                    callbacks=[callbacks_a, callbacks_b],
                    class_weight=class_weights)

print('##########################################################################')
print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")
print(Normalization_File)
print(Model_FileName)
#1735/1735 [==============================] - 2172s 1s/step - loss: 0.6249 - accuracy: 0.6511 - val_loss: 0.6145 - val_accuracy: 0.6602
big_model=model


Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_146 (Dense)           (None, 2500)              517500    
                                                                 
 batch_normalization_53 (Bat  (None, 2500)             10000     
 chNormalization)                                                
                                                                 
 dropout_53 (Dropout)        (None, 2500)              0         
                                                                 
 dense_147 (Dense)           (None, 600)               1500600   
                                                                 
 batch_normalization_54 (Bat  (None, 600)              2400      
 chNormalization)                                                
                                                                 
 dropout_54 (Dropout)        (None, 600)             

KeyboardInterrupt: 

In [ ]:
#ChatGPT2
from keras.layers import Dense, Dropout,BatchNormalization
from keras.models import Sequential
from keras.optimizers import Nadam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import regularizers
gc.collect()
# Model Optimization
SizeTunner=0.5
IN_DIM = dt.shape[1] - 1
model = Sequential()
model.add(Dense(int(200*SizeTunner), input_dim=IN_DIM, activation='elu' , activity_regularizer=regularizers.l1(0.01)))
model.add(BatchNormalization())
model.add(Dense(int(80*SizeTunner), activation='elu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(int(40*SizeTunner), activation='elu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(int(20*SizeTunner), activation='sigmoid'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(int(50*SizeTunner), activation='elu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(int(10*SizeTunner), activation='elu'))
model.add(BatchNormalization())

model.add(Dense(1, activation='sigmoid'))

print(model.summary())

optimizer = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

callbacks_a = ModelCheckpoint(filepath=Model_FileName, monitor='val_accuracy', save_best_only=True, save_weights=True)
callbacks_b = EarlyStopping(monitor='val_accuracy', mode='auto', patience=8, verbose=1)

print("saving file in: " + Model_FileName)
history = model.fit(dt[index_20percent:, :-1],
                    dt[index_20percent:, -1],
                    validation_data=(dt[:index_20percent, :-1], dt[:index_20percent, -1]),
                    epochs=500,
                    batch_size=256*20,
                    callbacks=[callbacks_a, callbacks_b])

In [ ]:
import gc
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Nadam
from keras.callbacks import ModelCheckpoint, EarlyStopping
# Define the class weights
class_weights = {0: 1., 1: 1.}

gc.collect()

SizeTunner = 1
IN_DIM = dt.shape[1] - 1

model = Sequential()
model.add(Dense(int(200 * SizeTunner), input_dim=IN_DIM, activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(int(80 * SizeTunner), activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(int(80 * SizeTunner), activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(int(80 * SizeTunner), activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(int(80 * SizeTunner), activation='elu'))
model.add(BatchNormalization())
model.add(Dense(int(80 * SizeTunner), activation='elu'))
model.add(BatchNormalization())
model.add(Dense(int(80 * SizeTunner), activation='relu'))
model.add(BatchNormalization())
model.add(Dense(int(20 * SizeTunner), activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

optimizer = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

callbacks = [
    ModelCheckpoint(filepath=Model_FileName, monitor='val_accuracy', save_best_only=True, save_weights=True),
    EarlyStopping(monitor='val_accuracy', mode='auto', patience=6, verbose=1)
]

print("saving file in: " + Model_FileName)
history = model.fit(dt[index_20percent:, :-1],
                    dt[index_20percent:, -1],
                    validation_data=(dt[:index_20percent, :-1], dt[:index_20percent, -1]),
                    epochs=500,
                    batch_size=256*50,
                    callbacks=callbacks,
                    class_weight=class_weights)

#868/868 [==============================] - 30s 35ms/step - loss: 0.6078 - accuracy: 0.6664 - val_loss: 0.6107 - val_accuracy: 0.6639 >0.6646
#Results after 380 min
# Epoch 133/500
# 347/347 [==============================] - 138s 398ms/step - loss: 0.5867 - accuracy: 0.6842 - val_loss: 0.5839 - val_accuracy: 0.6863
# Epoch 134/500
# 347/347 [==============================] - 137s 395ms/step - loss: 0.5865 - accuracy: 0.6843 - val_loss: 0.5845 - val_accuracy: 0.6861
# Epoch 134: early stopping



# Saving the model

In [146]:
very_deep_model=model

In [145]:
model.save(f'{DATA_DIR}/tp{int(BUY_PERCENT*100)}_w{WINDOW_SIZE}_max{MAX_FORCAST_SIZE}min_Model'+"_VeryDeep.h5")

In [ ]:
model.compile()

In [67]:
model=load_model(Model_FileName+"_v2.h5")
model1=load_model(Model_FileName+"_v1.h5")

In [ ]:
mini_range_start=0
mini_range_stop=200000
model.evaluate(dt[mini_range_start:mini_range_stop,:-1],dt[mini_range_start:mini_range_stop,-1])

# Unsing CNN

In [ ]:
from keras.applications.vgg16 import VGG16

# Load the VGG16 model without the top layer (i.e., without the final classification layer)
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(IN_DIM,))

# Freeze all layers in the VGG16 model
for layer in vgg16.layers:
    layer.trainable = False

# Add the VGG16 model to the hidden layers of your network
model = Sequential()
model.add(vgg16)
model.add(Flatten())
model.add(Dense(int(200), activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(int(60), activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(int(30), activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(int(10), activation='sigmoid'))
model.add(Dense(int(30), activation='elu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
callbacks_a = ModelCheckpoint(filepath=Model_FileName, monitor='val_accuracy', save_best_only=True, save_weights=True)
callbacks_b = EarlyStopping(monitor='val_accuracy', mode='auto', patience=8, verbose=1)
history = model.fit(dt[index_20percent:, 0:-1],
                dt[index_20percent:, -1],
                validation_data=(dt[:index_20percent, :-1], dt[:index_20percent, -1]),
                epochs=500,
                batch_size=256 * 10,
                callbacks=[callbacks_a, callbacks_b])

# Print results
print('##########################################################################')
print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")
print(Normalization_File)
print(Model_FileName)


## 3-  Test

### stats plus

In [148]:
USED_MODEL=very_deep_model
Prediction_Note=USED_MODEL.predict( dt[:, 0:-1])
prediction2=Prediction_Note.round()

173438/173438 [==============================] - 492s 3ms/step


In [149]:
prediction2[:,0].mean()

0.47909352

In [150]:
PesemisticPrediction=(PredNote[:,0]-0.2).round()

In [151]:
PesemisticPrediction.mean()

0.15808685

In [152]:
Y=dt[:,-1].copy()
Y

array([1., 1., 1., ..., 0., 1., 0.], dtype=float32)

In [153]:
Pred01=prediction2[:,-1]
Pred01

array([0., 1., 0., ..., 0., 0., 0.], dtype=float32)

In [179]:
TruePred=((Y==Pred01)).copy()
ModelAccuracy=hp(TruePred.mean(),"ModelAccuracy")

TrueWinPred=((Pred01==1) & (Y==1) ).copy()
TrueWinPred_Mean=hp(TrueWinPred.mean(),"True Win Predictions Mean of all")

LossPred=((Pred01==1) & (Y==0) ).copy()
LossPred_Mean=hp(LossPred.mean(),"XXX Loss Buy Mean of all")

MissedDealPred=((Pred01==0) & (Y==1) ).copy()
MissedDeal_Mean=hp(MissedDealPred.mean(),"Missed good deal off all")

GoodZeroPred=((Pred01==0) & (Y==0) ).copy()
GoodZero_Mean=hp(GoodZeroPred.mean(),"Good Zero prediction Mean")

fiability=TrueWinPred_Mean + LossPred_Mean + MissedDeal_Mean + GoodZero_Mean
if( fiability == 100):print("good fiability")
else: print(f"check the fiability {fiability}")

ModelAccuracy: 69.178%
True Win Predictions Mean of all: 33.543%
XXX Loss Buy Mean of all: 14.366%
Missed good deal off all: 16.457%
Good Zero prediction Mean: 35.634%
good fiability


100.0

In [ ]:
TrueNoBuyPred=((Pred01==0) & (Y==0) ).copy()
hp(TrueNoBuyPred.mean())

In [116]:
ixx=dt.shape[0]

In [117]:
False_prediction=dt[:ixx,-1][(prediction2[:ixx].transpose()[0]!=dt[:ixx])]
True_prediction=dt[:ixx,-1][(prediction2[:ixx].transpose()[0]==dt[:ixx])]

In [118]:
dt[:,-1].mean()*100

50.0

In [119]:
fp0=False_prediction[False_prediction==0].shape[0]*100/ixx #### it mean that 11.73 of the 0 class are wrong the buyed witch lead the losses
print( f"Prediction mean :{prediction2.mean()*100} %"+'#### the colose to 50 the better')
print( f"False prediction class 0 :{fp0} %"+'#### it mean that x of the 0 class are wrong the buyed witch lead the losses')
fp1=False_prediction[False_prediction==1].shape[0]*100/ixx   #### we don't buy x % of the correct chanses
print( f"False prediction class 1 :{fp1} %"+'#### we don\'t buy x % of the correct chanses')
trp0=True_prediction[True_prediction==0].shape[0]*100/ixx   #### it mean that 40 % of class 0 are predicted correctly
trp1=True_prediction[True_prediction==1].shape[0]*100/ixx     #### the only buying correct of all
print( f"True prediction class 0 :{trp0} %"+'#### it mean that 40 % of class 0 are predicted correctly')
print( f"True prediction class 1 :{trp1} %"+'#### the only buying correct of all')
print(f"successful buy pourcent of unsuccessfull: {100*trp1/(trp1+fp1)}  %")


Prediction mean :50.47912001609802 %#### the colose to 50 the better
False prediction class 0 :50.0 %#### it mean that x of the 0 class are wrong the buyed witch lead the losses
False prediction class 1 :50.0 %#### we don't buy x % of the correct chanses
True prediction class 0 :0.0 %#### it mean that 40 % of class 0 are predicted correctly
True prediction class 1 :0.0 %#### the only buying correct of all
successful buy pourcent of unsuccessfull: 0.0  %


In [120]:
model.evaluate(dt[:index_20percent,:-1], Y[:index_20percent]) #3-> 66.76 / 4 -> 65.75


 8008/34688 [=====>........................] - ETA: 1:19 - loss: 0.7167 - accuracy: 0.6042

KeyboardInterrupt: 